In [89]:
import requests
import pandas as pd
from datetime import datetime
import json
import ast
import asyncio
import aiohttp
from IPython.core.display_functions import clear_output
from py_clob_client.clob_types import MarketOrderArgs, OrderType

In [93]:
def fetch_all_markets():
    markets = []
    offset = 0
    limit = 100
    while True:
        url = f"https://gamma-api.polymarket.com/markets?includeOrderBook=true&includePrices=true&limit={limit}&offset={offset}&closed=false&active=true"
        response = requests.get(url)
        if response.status_code != 200:
            raise Exception(f"Failed to fetch markets: {response.text}")
        data = response.json()
        if not data:
            break
        markets.extend(data)
        offset += limit
    return markets

In [134]:
raw = pd.DataFrame(fetch_all_markets())

In [135]:
raw

,id,question,conditionId,slug,resolutionSource,endDate,liquidity,startDate,image,icon,...,gameId,eventStartTime,line,volume24hrAmm,volume1wkAmm,volume1moAmm,volume1yrAmm,volumeAmm,liquidityAmm,oneYearPriceChange
0,516706,Fed rate hike in 2025?,0x4319532e181605cb15b1bd677759a3bc7f7394b2fdf1...,fed-rate-hike-in-2025,,2025-12-10T12:00:00Z,57656.2183,2024-12-29T22:50:33.584839Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,516710,US recession in 2025?,0xfa48a99317daef1654d5b03e30557c4222f276657275...,us-recession-in-2025,,2026-02-28T12:00:00Z,96390.6665,2025-01-08T01:33:54.924Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,516711,Fed emergency rate cut in 2025?,0xcddb4f52babca280f8e5fc4b9146f2520f0bfb88e134...,fed-emergency-rate-cut-in-2025,,2025-12-31T12:00:00Z,32328.25165,2024-12-29T22:41:48.24Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,516712,Tether insolvent in 2025?,0x1bbdc459d73e6972f5ba1f687d739fde70a9774f4875...,tether-insolvent-in-2025,,2025-12-31T12:00:00Z,33321.21255,2024-12-29T22:41:19.741Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,516713,USDT depeg in 2025?,0x416316490efec1038ce09ec0184f82f1f7921876ee05...,usdt-depeg-in-2025,,2025-12-31T12:00:00Z,117976.44689,2024-12-29T22:40:52.581438Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17092,879848,"XRP Up or Down - December 8, 5:30AM-5:45AM ET",0xf5b17da03ac3d943ef9b16be47a3f1c378f61aef3042...,xrp-updown-15m-1765189800,https://data.chain.link/streams/xrp-usd,2025-12-08T10:45:00Z,6292.8462,2025-12-07T10:32:48.855778Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,NaN,2025-12-08T10:30:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17093,879849,"Ethereum Up or Down - December 8, 5:45AM-6:00A...",0x6691bdc1984adf23f28979b44b8577654f11578a2eb1...,eth-updown-15m-1765190700,https://data.chain.link/streams/eth-usd,2025-12-08T11:00:00Z,0,2025-12-07T10:47:45.168694Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,NaN,2025-12-08T10:45:00Z,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17094,879850,"Bitcoin Up or Down - December 8, 5:45AM-6:00AM ET",0xcadd1efe474fb7e306b40011795a4cf4df34395551fe...,btc-updown-15m-1765190700,https://data.chain.link/streams/btc-usd,2025-12-08T11:00:00Z,0,2025-12-07T10:47:44.91402Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,NaN,2025-12-08T10:45:00Z,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17095,879851,"XRP Up or Down - December 8, 5:45AM-6:00AM ET",0x5d9d0597467f98b02d16529c50a8bcab17a396e6a8c9...,xrp-updown-15m-1765190700,https://data.chain.link/streams/xrp-usd,2025-12-08T11:00:00Z,0,2025-12-07T10:47:48.904618Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,NaN,2025-12-08T10:45:00Z,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [137]:
test = raw[raw['question'].str.startswith("Will Jesus")]
test

,id,question,conditionId,slug,resolutionSource,endDate,liquidity,startDate,image,icon,...,gameId,eventStartTime,line,volume24hrAmm,volume1wkAmm,volume1moAmm,volume1yrAmm,volumeAmm,liquidityAmm,oneYearPriceChange
328,529315,Will Jesus Christ return in 2025?,0xdb4d4cc853e23faf2b0e44277234052d83622d0785eb...,will-jesus-christ-return-in-2025,,2025-12-31T12:00:00Z,333031.88776,2025-03-20T22:05:28.102478Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572,540819,Will Jesus Christ return before GTA VI?,0x32b09f6390252b37d674501527e709016d55581b2c1e...,will-jesus-christ-return-before-gta-vi-665,,2026-07-31T12:00:00Z,569438.4814,2025-05-02T15:48:16.854Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8886,701012,Will Jesus Salate win the PDC World Darts Cham...,0x5c66663e748948d8b6322dda5413caeff1e25e987863...,will-jesus-salate-win-the-pdc-world-darts-cham...,NaN,2026-01-03T00:00:00Z,3872.10282,2025-11-24T18:11:31.869888Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9557,703258,Will Jesus Christ return before 2027?,0x0b4cc3b739e1dfe5d73274740e7308b6fb389c5af040...,will-jesus-christ-return-before-2027,,2026-12-31T00:00:00Z,43731.72216,2025-11-25T18:08:21.296603Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
df_epl = raw[raw["slug"].str.startswith("epl-")]
df_epl = df_epl[df_epl["question"].str.startswith("Will", na=False)]
df_epl["match_date"] = df_epl["slug"].apply(
    lambda s: "-".join(s.split("-")[3:6])
)

df_epl["match_date"] = pd.to_datetime(df_epl["match_date"])
today = pd.Timestamp.today().normalize()  # 去掉时分秒
df_epl = df_epl[df_epl["match_date"] >= today]

df_epl

,id,question,conditionId,slug,resolutionSource,endDate,liquidity,startDate,image,icon,...,negRiskMarketID,negRiskRequestID,umaResolutionStatus,deployingTimestamp,secondsDelay,sportsMarketType,gameId,eventStartTime,line,match_date
8791,699749,Will Brighton win on 2025-12-07?,0x5b4053b0a9dcc22d75ac045ae16ef1b7e7e6d141910f...,epl-bri-wes-2025-12-07-bri,https://www.premierleague.com/,2025-12-07T14:00:00Z,305968.8101,2025-11-24T05:09:37.265817Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,0x4a8c8e569ff09509089b1f380e25a678e88efc4dee3c...,0x8ce9812c34656a13bf9b69756c437909a4a269fcb7f1...,NaN,2025-11-24T05:00:37.718772Z,3.0,moneyline,NaN,NaN,NaN,2025-12-07
8792,699750,Will Brighton vs. West Ham end in a draw?,0x7e4cb8bed23b41fc46c5d76b3344221dca69077d07d0...,epl-bri-wes-2025-12-07-draw,https://www.premierleague.com/,2025-12-07T14:00:00Z,316923.8897,2025-11-24T05:09:36.75727Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,0x4a8c8e569ff09509089b1f380e25a678e88efc4dee3c...,0xd5f10f5995fe83efd1999017e85b47ec0744748dcc3f...,NaN,2025-11-24T05:00:37.722842Z,3.0,moneyline,NaN,NaN,NaN,2025-12-07
8793,699751,Will West Ham win on 2025-12-07?,0x5462693624b679052257f6dee758919935e345e72559...,epl-bri-wes-2025-12-07-wes,https://www.premierleague.com/,2025-12-07T14:00:00Z,332784.6411,2025-11-24T05:09:37.0114Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,0x4a8c8e569ff09509089b1f380e25a678e88efc4dee3c...,0x93d7c402ab4948dd6562b9d8dbe49ad7eef80b81537f...,NaN,2025-11-24T05:00:37.725567Z,3.0,moneyline,NaN,NaN,NaN,2025-12-07
8806,699765,Will Fulham win on 2025-12-07?,0x43a24261931d613313225880e6769222c774082adf7f...,epl-ful-cry-2025-12-07-ful,https://www.premierleague.com/,2025-12-07T16:30:00Z,296870.9827,2025-11-24T05:09:49.540039Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,0x6b9656f83d92bd4c6d56239bd4cdd0c0bf3c69e79638...,0x111a3c56cd6e1f05e9bad803eb9a14c1cb02c9188d64...,NaN,2025-11-24T05:00:56.940333Z,3.0,moneyline,NaN,NaN,NaN,2025-12-07
8807,699766,Will Fulham vs. Crystal Palace end in a draw?,0x141a7aa79af227a252632b812067b52464c7f2827caa...,epl-ful-cry-2025-12-07-draw,https://www.premierleague.com/,2025-12-07T16:30:00Z,308568.3164,2025-11-24T05:09:49.795485Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,0x6b9656f83d92bd4c6d56239bd4cdd0c0bf3c69e79638...,0x2db9ac15e004681db44b973f8df2a16ddd64fdaf711f...,NaN,2025-11-24T05:00:56.943649Z,3.0,moneyline,NaN,NaN,NaN,2025-12-07
8809,699768,Will Crystal Palace win on 2025-12-07?,0x7d2c7acaeeaf992d215dd6bd00053b300c9316be620d...,epl-ful-cry-2025-12-07-cry,https://www.premierleague.com/,2025-12-07T16:30:00Z,296909.7199,2025-11-24T05:09:50.823005Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,0x6b9656f83d92bd4c6d56239bd4cdd0c0bf3c69e79638...,0x2f8fd8079941e9183915477c86ced98c804342b6b2d4...,NaN,2025-11-24T05:00:56.946927Z,3.0,moneyline,NaN,NaN,NaN,2025-12-07
9431,702306,Will Wolves win on 2025-12-08?,0xe7ceaefb287d4fc475d053c891ae86d5143e430ff014...,epl-wol-mun-2025-12-08-wol,https://www.premierleague.com/,2025-12-08T20:00:00Z,308692.8473,2025-11-25T05:08:49.361454Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,0x25d93675349269a4ac2f4ab10678fc4e6836fbbf1ff4...,0xb58c3e0806dda59cc51ac5e1bfdb154015f0ea6277ef...,NaN,2025-11-25T05:00:43.352801Z,3.0,moneyline,NaN,NaN,NaN,2025-12-08
9435,702311,Will Wolves vs. Manchester United end in a draw?,0x8087668016019c5ea0fd41d2f99beba484fc45478dd8...,epl-wol-mun-2025-12-08-draw,https://www.premierleague.com/,2025-12-08T20:00:00Z,301743.6542,2025-11-25T05:08:48.852468Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,0x2

In [111]:
df_epl_ml = df_epl.head(3)
df_epl_ml

,id,question,conditionId,slug,resolutionSource,endDate,liquidity,startDate,image,icon,...,negRiskMarketID,negRiskRequestID,umaResolutionStatus,deployingTimestamp,secondsDelay,sportsMarketType,gameId,eventStartTime,line,match_date
8791,699749,Will Brighton win on 2025-12-07?,0x5b4053b0a9dcc22d75ac045ae16ef1b7e7e6d141910f...,epl-bri-wes-2025-12-07-bri,https://www.premierleague.com/,2025-12-07T14:00:00Z,305968.8101,2025-11-24T05:09:37.265817Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,0x4a8c8e569ff09509089b1f380e25a678e88efc4dee3c...,0x8ce9812c34656a13bf9b69756c437909a4a269fcb7f1...,NaN,2025-11-24T05:00:37.718772Z,3.0,moneyline,NaN,NaN,NaN,2025-12-07
8792,699750,Will Brighton vs. West Ham end in a draw?,0x7e4cb8bed23b41fc46c5d76b3344221dca69077d07d0...,epl-bri-wes-2025-12-07-draw,https://www.premierleague.com/,2025-12-07T14:00:00Z,316923.8897,2025-11-24T05:09:36.75727Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,0x4a8c8e569ff09509089b1f380e25a678e88efc4dee3c...,0xd5f10f5995fe83efd1999017e85b47ec0744748dcc3f...,NaN,2025-11-24T05:00:37.722842Z,3.0,moneyline,NaN,NaN,NaN,2025-12-07
8793,699751,Will West Ham win on 2025-12-07?,0x5462693624b679052257f6dee758919935e345e72559...,epl-bri-wes-2025-12-07-wes,https://www.premierleague.com/,2025-12-07T14:00:00Z,332784.6411,2025-11-24T05:09:37.0114Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,0x4a8c8e569ff09509089b1f380e25a678e88efc4dee3c...,0x93d7c402ab4948dd6562b9d8dbe49ad7eef80b81537f...,NaN,2025-11-24T05:00:37.725567Z,3.0,moneyline,NaN,NaN,NaN,2025-12-07


In [112]:
df_epl_ml["clobTokenIds"] = df_epl_ml["clobTokenIds"].apply(ast.literal_eval)
df_epl_ml["yes_token"] = df_epl_ml["clobTokenIds"].apply(lambda x: x[0])
df_epl_ml["no_token"]  = df_epl_ml["clobTokenIds"].apply(lambda x: x[1])

/var/folders/nb/lml3l_ls6b7gzy20lpv559vh0000gn/T/ipykernel_43761/1457815985.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epl_ml["clobTokenIds"] = df_epl_ml["clobTokenIds"].apply(ast.literal_eval)
/var/folders/nb/lml3l_ls6b7gzy20lpv559vh0000gn/T/ipykernel_43761/1457815985.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epl_ml["yes_token"] = df_epl_ml["clobTokenIds"].apply(lambda x: x[0])
/var/folders/nb/lml3l_ls6b7gzy20lpv559vh0000gn/T/ipykernel_43761/1457815985.py:3: SettingWithCopyWarning:

In [113]:
yes_tokens = df_epl_ml["yes_token"].tolist()
df_epl_ml

,id,question,conditionId,slug,resolutionSource,endDate,liquidity,startDate,image,icon,...,umaResolutionStatus,deployingTimestamp,secondsDelay,sportsMarketType,gameId,eventStartTime,line,match_date,yes_token,no_token
8791,699749,Will Brighton win on 2025-12-07?,0x5b4053b0a9dcc22d75ac045ae16ef1b7e7e6d141910f...,epl-bri-wes-2025-12-07-bri,https://www.premierleague.com/,2025-12-07T14:00:00Z,305968.8101,2025-11-24T05:09:37.265817Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,NaN,2025-11-24T05:00:37.718772Z,3.0,moneyline,NaN,NaN,NaN,2025-12-07,9211726791532092639988167245301533995991830936...,9921530920243205253326614601184885088865604960...
8792,699750,Will Brighton vs. West Ham end in a draw?,0x7e4cb8bed23b41fc46c5d76b3344221dca69077d07d0...,epl-bri-wes-2025-12-07-draw,https://www.premierleague.com/,2025-12-07T14:00:00Z,316923.8897,2025-11-24T05:09:36.75727Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,NaN,2025-11-24T05:00:37.722842Z,3.0,moneyline,NaN,NaN,NaN,2025-12-07,9186278757666359311082800401838919802837265744...,5564593962438319470475424788823144507242927263...
8793,699751,Will West Ham win on 2025-12-07?,0x5462693624b679052257f6dee758919935e345e72559...,epl-bri-wes-2025-12-07-wes,https://www.premierleague.com/,2025-12-07T14:00:00Z,332784.6411,2025-11-24T05:09:37.0114Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,...,NaN,2025-11-24T05:00:37.725567Z,3.0,moneyline,NaN,NaN,NaN,2025-12-07,2507525369672551193063574368331984829543294741...,4607077232521919563416575960944090861658553156...


In [121]:
tokens = []
for idx, row in df_epl_ml.iterrows():
    tok = row['yes_token']
    tokens.append(tok)
tokens

['92117267915320926399881672453015339959918309365273899216398653776846773774985',
 '91862787576663593110828004018389198028372657447052365064742910695901241302500',
 '25075253696725511930635743683319848295432947418295991212388362018388075099395']

In [46]:
def get_pol_book(token_id):
    url = "https://clob.polymarket.com/book"
    return requests.get(url, params={"token_id": token_id}).json()

In [123]:
BASE_URL = "https://clob.polymarket.com/book"

In [88]:
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import OrderArgs, OrderType
from py_clob_client.order_builder.constants import BUY

host: str = "https://clob.polymarket.com"
key: str = "0x644dec691ec2cabe992ee0c10d1249e391492c83d340e49a2ae82de94b97f262" #This is your Private Key. Export from https://reveal.magic.link/polymarket or from your Web3 Extension
chain_id: int = 137 #No need to adjust this
POLYMARKET_PROXY_ADDRESS: str = "0x5BB1D23128cd2bF3F60ba48405D3c6aa5f3549fa" #This is the address listed below your profile picture when using the Polymarket site.

#Select from the following 3 initialization options to match your login method, and remove any unused lines so only one client is initialized.


### Initialization of a client using a Polymarket Proxy associated with an Email/Magic account. If you login with your email use this example.
client = ClobClient(host, key=key, chain_id=chain_id, signature_type=1, funder=POLYMARKET_PROXY_ADDRESS)

### Initialization of a client using a Polymarket Proxy associated with a Browser Wallet(Metamask, Coinbase Wallet, etc)
#
### Initialization of a client that trades directly from an EOA. (If you don't know what this means, you're not using it)
#client = ClobClient(host, key=key, chain_id=chain_id)

## Create and sign a limit order buying 5 tokens for 0.010c each
#Refer to the API documentation to locate a tokenID: https://docs.polymarket.com/developers/gamma-markets-api/fetch-markets-guide

client.set_api_creds(client.create_or_derive_api_creds())

In [87]:
def place_order(token_id,size,price):
    order_args = OrderArgs(
    price=price,
    size=size,
    side=BUY,
    token_id= token_id
    )
    signed_order = client.create_order(order_args)

    ## GTC(Good-Till-Cancelled) Order
    resp = client.post_order(signed_order)
    print(resp)

In [91]:
def place_market_order(token_id,size):

    market_order = client.create_market_order(MarketOrderArgs(
        token_id=token_id,
        amount=size,
        side=BUY
    ))
    resp = client.post_order(market_order, OrderType.FOK)
    print(resp)


In [126]:
def now():
    return datetime.datetime.now().strftime("%H:%M:%S")

def print_line(txt):
    print("\r" + txt, end="", flush=True)


async def get_pol_book(session, token_id):
    async with session.get(BASE_URL, params={"token_id": token_id}) as resp:
        return await resp.json()

def best_ask_and_size(book):
    try:
        level = book["asks"][-1]
        return float(level["price"]), float(level["size"])
    except:
        return None, None


async def main(tokens):

    async with aiohttp.ClientSession() as session:

        count = 0
        while True:

            # 异步并发获取三个订单簿
            book_home, book_draw, book_guest = await asyncio.gather(
                get_pol_book(session, tokens[0]),
                get_pol_book(session, tokens[1]),
                get_pol_book(session, tokens[2]),
            )

            # best ask + size
            ask_home, size_home = best_ask_and_size(book_home)
            ask_draw, size_draw = best_ask_and_size(book_draw)
            ask_guest, size_guest = best_ask_and_size(book_guest)

            if None in [ask_home, ask_draw, ask_guest]:
                await asyncio.sleep(0.5)
                continue

            # 套利价格合计
            s = ask_home + ask_draw + ask_guest

            # 现实容量 = 三条腿中 best ask size 最小值
            capacity = min(size_home, size_draw, size_guest)
            ts = time.strftime('%H:%M:%S')
            if s < 1:
                profit = 1/s - 1
                print(f"{ts}|")
                print("\n🚀🚀🚀 发现套利机会！")
                print(f"  SUM={s:.6f}")
                print(f"  现实容量: {capacity:.1f} shares")
                print(f"  无风险收益率: {profit*100:.3f}%\n")
                print(f"----------------")

                record[ts] = s
                capa[ts] = capacity
                count += 1

            print_line(f"{ts}| Home={ask_home:.4f} (sz={size_home:.1f}), "
                 f"DRAW={ask_draw:.4f} (sz={size_draw:.1f}), "
                      f"Guest={ask_guest:.4f} (sz={size_guest:.1f}), SUM={s:.4f}, CAP={capacity:.1f}")




            await asyncio.sleep(0.01)


In [129]:
record = {}
capa = {}
await(main(tokens))

10:37:07| Home=0.6200 (sz=97796.2), DRAW=0.2300 (sz=97881.8), Guest=0.1800 (sz=37272.7), SUM=1.0300, CAP=37272.7

CancelledError: 

In [84]:
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import OrderArgs, OrderType
from py_clob_client.order_builder.constants import BUY

host: str = "https://clob.polymarket.com"
key: str = "0x644dec691ec2cabe992ee0c10d1249e391492c83d340e49a2ae82de94b97f262" #This is your Private Key. Export from https://reveal.magic.link/polymarket or from your Web3 Extension
chain_id: int = 137 #No need to adjust this
POLYMARKET_PROXY_ADDRESS: str = "0x5BB1D23128cd2bF3F60ba48405D3c6aa5f3549fa" #This is the address listed below your profile picture when using the Polymarket site.

#Select from the following 3 initialization options to match your login method, and remove any unused lines so only one client is initialized.


### Initialization of a client using a Polymarket Proxy associated with an Email/Magic account. If you login with your email use this example.
client = ClobClient(host, key=key, chain_id=chain_id, signature_type=1, funder=POLYMARKET_PROXY_ADDRESS)

### Initialization of a client using a Polymarket Proxy associated with a Browser Wallet(Metamask, Coinbase Wallet, etc)
#
### Initialization of a client that trades directly from an EOA. (If you don't know what this means, you're not using it)
#client = ClobClient(host, key=key, chain_id=chain_id)

## Create and sign a limit order buying 5 tokens for 0.010c each
#Refer to the API documentation to locate a tokenID: https://docs.polymarket.com/developers/gamma-markets-api/fetch-markets-guide

client.set_api_creds(client.create_or_derive_api_creds())


In [85]:
client

In [ ]:
def place_order(token_id,size,price):
    order_args = OrderArgs(
    price=price,
    size=size,
    side=BUY,
    token_id= token_id #Token ID you want to purchase goes here. Example token: 114304586861386186441621124384163963092522056897081085884483958561365015034812 ( Xi Jinping out in 2025, YES side )
    )
    signed_order = client.create_order(order_args)

    ## GTC(Good-Till-Cancelled) Order
    resp = client.post_order(signed_order)
    print(resp)


In [86]:
order_args = OrderArgs(
    price=0.996,
    size=5.0,
    side=BUY,
    token_id= '43346999573571279084829705550951657228560122189852907354399832194708460277206' #Token ID you want to purchase goes here. Example token: 114304586861386186441621124384163963092522056897081085884483958561365015034812 ( Xi Jinping out in 2025, YES side )
)
signed_order = client.create_order(order_args)

## GTC(Good-Till-Cancelled) Order
resp = client.post_order(signed_order)
print(resp)

{'errorMsg': '', 'orderID': '0x11e40f35100576a782aea879a654366255347b19e9c5fb5bc65a11e6ee35067c', 'takingAmount': '5', 'makingAmount': '4.98', 'status': 'matched', 'transactionsHashes': ['0x265ccf73c650cbbb9b95ba56ad8e2e94e6706a523658eebd1d94d4aac3f3ff04'], 'success': True}


In [75]:
from py_clob_client.client import ClobClient
import os

# Initialize with just your private key
client = ClobClient(
    host="https://clob.polymarket.com",
    chain_id=137,
    key="0x644dec691ec2cabe992ee0c10d1249e391492c83d340e49a2ae82de94b97f262"
)

# Try to create or derive API credentials (tests L1 signing)
try:
    api_creds = client.derive_api_key()
    print("✓ Private key signing works!")
    print(f"API Key: {api_creds.api_key}")
except Exception as e:
    print(f"✗ Signing failed: {e}")

✓ Private key signing works!
API Key: 0f2eedfb-aad7-79cb-d00c-62317f40e214


In [54]:
get_pol_book(43346999573571279084829705550951657228560122189852907354399832194708460277206)

{'market': '0xdb4d4cc853e23faf2b0e44277234052d83622d0785eb1e37c3584f574383461c',
 'asset_id': '43346999573571279084829705550951657228560122189852907354399832194708460277206',
 'timestamp': '1765115469763',
 'hash': '7dbcfc57dd4fca520f7ef1f949680089860c9335',
 'bids': [{'price': '0.001', 'size': '7099451.22'},
  {'price': '0.002', 'size': '524288'},
  {'price': '0.003', 'size': '5'},
  {'price': '0.004', 'size': '131072'},
  {'price': '0.005', 'size': '300000'},
  {'price': '0.006', 'size': '8.86'},
  {'price': '0.008', 'size': '32768'},
  {'price': '0.009', 'size': '90000'},
  {'price': '0.01', 'size': '500005'},
  {'price': '0.016', 'size': '8192'},
  {'price': '0.017', 'size': '9.11'},
  {'price': '0.032', 'size': '2048'},
  {'price': '0.033', 'size': '100000'},
  {'price': '0.037', 'size': '299999'},
  {'price': '0.038', 'size': '100000'},
  {'price': '0.051', 'size': '200156.92'},
  {'price': '0.052', 'size': '104.03'},
  {'price': '0.053', 'size': '14.46'},
  {'price': '0.064', 's

In [239]:
from py_clob_client.clob_types import OrderArgs
from py_clob_client.order_builder.constants import BUY

# Place a limit order in one step
response = await client.create_and_post_order(
    OrderArgs(
        token_id="43346999573571279084829705550951657228560122189852907354399832194708460277206",  # Get from Gamma API
        price=0.996,                # Price per share
        size=5,                   # Number of shares
        side=BUY,                  # or SELL
    )
)

print(f"Order placed! ID: {response['orderID']}")

PolyApiException: PolyApiException[status_code=400, error_message={'error': 'invalid signature'}]

In [51]:
from py_clob_client.client import ClobClient

host: str = "https://clob.polymarket.com"
key: str = "0x644dec691ec2cabe992ee0c10d1249e391492c83d340e49a2ae82de94b97f262" #This is your Private Key. If using email login export from https://reveal.magic.link/polymarket otherwise export from your Web3 Application
chain_id: int = 137 #No need to adjust this
POLYMARKET_PROXY_ADDRESS: str = '0x0ad50eeb442c1cc4d193c2d91554afc60cee7f82' #This is the address you deposit/send USDC to to FUND your Polymarket account.

#Select from the following 3 initialization options to matches your login method, and remove any unused lines so only one client is initialized.

### Initialization of a client using a Polymarket Proxy associated with an Email/Magic account. If you login with your email use this example.
#client = ClobClient(host, key=key, chain_id=chain_id, signature_type=1, funder=POLYMARKET_PROXY_ADDRESS)

### Initialization of a client using a Polymarket Proxy associated with a Browser Wallet(Metamask, Coinbase Wallet, etc)
client = ClobClient(host, key=key, chain_id=chain_id, signature_type=2, funder=POLYMARKET_PROXY_ADDRESS)

### Initialization of a client that trades directly from an EOA.
#client = ClobClient(host, key=key, chain_id=chain_id)

print( client.derive_api_key() )

ApiCreds(api_key='0f2eedfb-aad7-79cb-d00c-62317f40e214', api_secret='qVGmZoLWoyr0LrcVpym-tDOV6YSLDtbNY--Lc7thp6Y=', api_passphrase='be65f4284dbaf1d81f0af3e0e2f2ab19f66d1f7e89325726d8bad694ec3174b2')


In [58]:
from eth_account import Account
account = Account.from_key("0x644dec691ec2cabe992ee0c10d1249e391492c83d340e49a2ae82de94b97f262")
print(f"Your address: {account.address}")

Your address: 0x58F554A15eD33eb2a3311C98535911Ed3c137eAF


In [24]:
from py_clob_client.client import ClobClient
import os

host = "https://clob.polymarket.com"
chain_id = 137 # Polygon mainnet
private_key = "0x644dec691ec2cabe992ee0c10d1249e391492c83d340e49a2ae82de94b97f262"

client = ClobClient(
    host="https://clob.polymarket.com",
    chain_id=137,
    key="0x644dec691ec2cabe992ee0c10d1249e391492c83d340e49a2ae82de94b97f262",
    creds=api_creds,  # Generated from L1 auth, API credentials enable L2 methods
    signature_type=2,  # signatureType explained below
    funder="0x0ad50eeb442c1cc4d193c2d91554afc60cee7f82" # funder explained below
)


In [33]:
api_creds

{'api_key': '0f2eedfb-aad7-79cb-d00c-62317f40e214',
 'secret': 'qVGmZoLWoyr0LrcVpym-tDOV6YSLDtbNY--Lc7thp6Y=',
 'passphrase': 'be65f4284dbaf1d81f0af3e0e2f2ab19f66d1f7e89325726d8bad694ec3174b2'}

In [80]:
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import OrderArgs
from py_clob_client.order_builder.constants import BUY
import os

# Initialize
client = ClobClient(
    host="https://clob.polymarket.com",
    chain_id=137,
    key="Loki827864",
    signature_type=2,  # 2=Gnosis Safe, 1=Magic Link
    funder='0x0ad50eeb442c1cc4d193c2d91554afc60cee7f82'
)

# Set API credentials
client.set_api_creds(client.create_or_derive_api_creds())

# Place order
resp = client.create_and_post_order(
    OrderArgs(
        token_id="43346999573571279084829705550951657228560122189852907354399832194708460277206",
        price=0.996,
        size=5,
        side=BUY
    )
)
print(resp)

Error: Non-hexadecimal digit found